# Running an Individual Test Plan

This notebook shows how to run an individual test plan and pass custom config parameters for the tests.

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [ ]:
%pip install --upgrade validmind

## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="..."
)

In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Load the Demo Dataset

In [ ]:
# You can also import customer_churn like this:
from validmind.datasets.classification import customer_churn as demo_dataset
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### Prepocess the Raw Dataset
We will need to preprocess the dataset and produce the training, test and validation splits first.

In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## Train a Model for Testing

We train a simple customer churn model for our test.

In [ ]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## Import and Run the Individual Test Plan

### Initialize ValidMind objects

We initize the objects required to run test plans using the ValidMind framework 

In [ ]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

### List of available Test Plans
The interface to show list of test plans available in the ValidMind development framework

In [ ]:
vm.test_plans.list_plans()


### Detail of an individual Test Plan
The interface will get detail of a specific test plan

In [ ]:
vm.test_plans.describe_plan("binary_classifier_model_diagnosis")

### Define the required config parameters
The config can be apply to specific test to override the default configuration parameters.

The format of a config is:
```
config = {
    "<test1_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
     "<test2_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
}
```

Users can input the configuration to test plan using **`config`**, allowing fine-tuning the suite according to their specific data requirements. 

In [ ]:
config={
    "overfit_regions": {
        "cut_off_percentage": 3,
        "feature_columns": ["Age", "Balance", "Tenure", "NumOfProducts"]
    },
    "weak_spots":{
        "features_columns": ["Age", "Balance"],
        "accuracy_gap_threshold": 85,
    },
    "robustness":{
        "features_columns": [ "Balance", "Tenure"],
        "scaling_factor_std_dev_list": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        "accuracy_decay_threshold": 4,
    }
}

### Run the test plan and display results

In [ ]:
model_diagnosis_test_plan = vm.run_test_plan("binary_classifier_model_diagnosis", 
                                             model=vm_model,
                                             config=config)

### Accessing the test plan results

We can now access all the results of the test plan, including subtest plans using `test_plan.get_results()`.

- `test_plan.get_results()`: With no arguments, this returns a list of all results
- `test_plan.get_results(test_id)`: If provided with a test id, this returns the all results that match the given test id

By default, get_results() returns a list, in case there are multiple tests with the same id.

In [ ]:
model_diagnosis_test_plan.get_results()

In [ ]:
model_robustness = model_diagnosis_test_plan.get_results("robustness")[0]
model_robustness.show()